In [44]:
with open('shakespear.txt', 'r', encoding='utf-8') as f:
    textToLearn = f.read()

In [45]:
tokens = sorted(list(set(textToLearn)))
vocab_size = len(tokens)
print(''.join(tokens))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [46]:
# iterate over tokens to create a dictionnary
#character:integer define the key/value of the dictionnary
encode_string_to_integer = {character:integer for integer,character in enumerate(tokens)}
decode_integer_to_string = {integer:character for integer,character in enumerate(tokens)}

tokenize = lambda string: [encode_string_to_integer[character] for character in string]
detokenize = lambda integers: ''.join([decode_integer_to_string[integer] for integer in integers])


print(tokenize("hello !!"))
print(detokenize(tokenize("hello !!")))

[46, 43, 50, 50, 53, 1, 2, 2]
hello !!


In [47]:
import torch
total_tokenized_data = torch.tensor(tokenize(textToLearn), dtype=torch.long)
print(total_tokenized_data.shape, total_tokenized_data.dtype)
print(total_tokenized_data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [48]:
training_set_length = int(0.9*len(total_tokenized_data))
training_set = total_tokenized_data[:training_set_length]
evaluation_set = total_tokenized_data[training_set_length:]

In [49]:
context_length = 8
training_set[:context_length+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [50]:
tokens_context = training_set[:context_length]
tokens_to_predict = training_set[1:context_length+1]
for i in range(context_length):
    current_context = tokens_context[:i+1]
    current_token_to_predict = tokens_to_predict[i]
    print(f"when input is {current_context} the solution is: {current_token_to_predict}")

when input is tensor([18]) the solution is: 47
when input is tensor([18, 47]) the solution is: 56
when input is tensor([18, 47, 56]) the solution is: 57
when input is tensor([18, 47, 56, 57]) the solution is: 58
when input is tensor([18, 47, 56, 57, 58]) the solution is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the solution is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the solution is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the solution is: 58


In [51]:
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(is_training):
    data_to_batch_from = training_set if is_training else evaluation_set
    # We generate random offsets to have a random chunk of text by batch
    random_offsets = torch.randint(len(data_to_batch_from) - context_length, (batch_size,))
    # We get random chunk of texts for each batch
    context_tokens = torch.stack([data_to_batch_from[offset:offset+context_length] for offset in random_offsets])
    solution_tokens = torch.stack([data_to_batch_from[offset+1:offset+context_length+1] for offset in random_offsets])
    return context_tokens, solution_tokens

batched_contexts, batched_solutions = get_batch(True)

In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, context_tokens, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(context_tokens) # (B,T,C)

        if targets is None:
            loss = None
        else:
            batch_size, context_length, channel_size = logits.shape
            logits = logits.view(batch_size*context_length, channel_size)
            targets = targets.view(batch_size*context_length)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, current_context_tokens, max_new_token_number):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_token_number):
            # get the predictions
            logits, loss = self(current_context_tokens)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probability_distribution = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            next_token = torch.multinomial(probability_distribution, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            current_context_tokens = torch.cat((current_context_tokens, next_token), dim=1) # (B, T+1)
        return current_context_tokens
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(batched_contexts, batched_solutions)
# We initialize with a new line character
line_character = torch.zeros((1,1), dtype=torch.long)
print(detokenize(m.generate(current_context_tokens = line_character, max_new_token_number =100)[0].tolist()))



Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [54]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [55]:
batch_size = 32
for steps in range(100000):
    batched_context, batched_solution = get_batch(True)

    logits, loss = m(batched_context, batched_solution)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

4.704006195068359
4.721118927001953
4.653193473815918
4.706261157989502
4.780904293060303
4.751267433166504
4.8395490646362305
4.667973041534424
4.743716716766357
4.774043083190918
4.6908278465271
4.789142608642578
4.61777925491333
4.650947093963623
4.886447429656982
4.703796863555908
4.757591724395752
4.65510892868042
4.709283828735352
4.6745147705078125
4.760501384735107
4.7892632484436035
4.653748512268066
4.6619181632995605
4.673007488250732
4.665776252746582
4.7301106452941895
4.755304336547852
4.712186813354492
4.745501518249512
4.726755619049072
4.735108375549316
4.777461051940918
4.643350601196289
4.6651835441589355
4.79764461517334
4.717412948608398
4.683647155761719
4.81886100769043
4.613771915435791
4.573785781860352
4.560741901397705
4.81563138961792
4.6061553955078125
4.619696140289307
4.725419521331787
4.650487899780273
4.5941481590271
4.7202863693237305
4.699342250823975
4.6724138259887695
4.727972507476807
4.66152286529541
4.616766929626465
4.599857807159424
4.653340339

In [56]:
print(detokenize(m.generate(current_context_tokens = line_character, max_new_token_number =100)[0].tolist()))


Bere ak hongacl GELusey wssisean thi's.
When, w t med yfthon oklichay ol, I INI' dede blais cicen m 
